### Refs
http://python-future.org/compatible_idioms.html

magic functions

* 与运算无关

|类别方法名|方法名
|----|----|
|字符串 /字节序列表示形式 | \__repr\__、\__str\__、\__format\__、\__bytes\__
|数值转换 | \__abs\__、\__bool\__、\__complex\__、\__int\__、\__float\__、\__hash\__、\__index\__
| 集合模拟 | \__len\__、\__getitem\__、\__setitem\__、\__delitem\__、\__contains\__
| 迭代枚举 | \__iter\__、\__reversed\__、\__next\__
| 可调用模拟 | \__call\__
|上下文管理 | \__enter\__、\__exit\__
| 实例创建和销毁 | \__new\__、\__init\__、\__del\__
|属性管理 | \__getattr\__、\__getattribute\__、\__setattr\__、\__delattr\__、\__dir\__
| 属性描述符 | \__get\__、\__set\__、\__delete\__
| 跟类相关的服务 | \__prepare\__、\__instancecheck\__、\__subclasscheck\__

* 与运算有关

|类别方法名|方法名
|----|----|
|一元运算符 | \__neg\__ -、\__pos\__ +、\__abs\__ abs()
|众多比较运算符 | \__lt\__ <、\__le\__ <=、\__eq\__ ==、\__ne\__ !=、\__gt\__ >、\__ge\__ >=
| 算术运算符 | \__add\__ +、\__sub\__ -、\__mul\__ *、\__truediv\__ /、\__floordiv\__ //、\__mod\__ %、\__divmod\__ divmod()、\__pow\__ ** 或pow()、\__round\__ round()
|反向算术运算符 | \__radd\__、\__rsub\__、\__rmul\__、\__rtruediv\__、\__rfloordiv\__、\__rmod\__、\__rdivmod\__、\__rpow\__
| 增量赋值算术运算符 | \__iadd\__、\__isub\__、\__imul\__、\__itruediv\__、\__ifloordiv\__、\__imod\__、\__ipow\__
| 位运算符 | \__invert\__ ~、\__lshift\__ <<、\__rshift\__ >>、\__and\__ &、\__or\__ |、\__xor\__ ^
| 反向位运算符 | \__rlshift\__、\__rrshift\__、\__rand\__、\__rxor\__、\__ror\__
|增量赋值位运算符|\__ilshift\__、\__irshift\__、\__iand\__、\__ixor\__、\__ior\__

### Term 4 - Style
短小的辅助函数比一味紧凑好

#### 从url中解码查询字符串

In [1]:
from urllib.parse import parse_qs
# url = 'https://cn.bing.com/search?red=5&blue=0&green='
values = parse_qs('red=5&blue=0&green=',
                 keep_blank_values=True)
# values = {'blue': ['0'], 'red': ['5'], 'green': ['']}

* 紧凑的写法
    * or, and -- bool表达式

In [2]:
red = values.get("red", [''])[0] or 0

* better: 用if else 表达式

In [3]:
red = values.get("red", [''])[0] if values.get("red") else 0

* even better: 辅助函数

In [4]:
def get_first_int(values, key, default=0):
    found = valud.get(key, [''])
    if found[0]:
        found = int(found[0])
    else:
        found = default
    return found

### Term 5 -  列表
使用slice，slice支持所有支持 \__getitem\__ 以及 \__setitem\__ 的类

In [5]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print('First Four: ', a[:4])
print('Last Four: ', a[-4:])

First Four:  ['a', 'b', 'c', 'd']
Last Four:  ['e', 'f', 'g', 'h']


* 使用slice时即使数组越界也ok， 但是访问单个元素越界会异常

In [6]:
a[:20]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

* slice操作会产生一个对原对象的**拷贝**， 而赋值仅仅是传递一个引用

In [7]:
b = a
c = a[:]
b.append('i')
print(a==b, a==c)

True False


### Term 6 -  列表
slice还可以有step（stride）参数，但不要给slice三个参数，step参数的使用有很多坑

In [8]:
to_reverse = b'abcdefg'
print('Bytes works fine: ', to_reverse[::-1])

Bytes works fine:  b'gfedcba'


In [9]:
to_reverse = u'哈利路亚'
print('Unicode works fine after 3.5: ', to_reverse[::-1])  # 3.4 sucks

Unicode works fine after 3.5:  亚路利哈


* 过多的复杂的start，end，stride操作降低代码可读性


In [10]:
a[-2:2:-2]    # what does -2,2,-2 mean

['h', 'f', 'd']

* 先slice，再step做两次，提高可读性。step尽量不用负数

### Term 7 -  列表
用list comprehension代替map，filter
* l2, l3等价，对于map而言如果是有条件的map，还要辅助filter

In [1]:
nums = [1, 2, 3, 4, 5]
l2 = [x*2 for x in l if x % 2 == 0]
l3 = map(lambda x: x*2, filter(lambda x: x%2 ==0, nums))
list(l3) == l2

True

In [2]:
[x * 2 for x in nums if x % 2 == 0]

[4, 8]

### Term 8 - 列表
list comprehension虽好，不要用嵌套

### Term 9 - Iterator （遍历）
使用生成式表达式来处理较大的list comprehension，来解决效率问题

使用列表推导的一个潜在缺陷是如果输入非常大的时候会产成一个很大的结果集，占用大量内存。
* list comprehension每一次调用，都会把返回值存在内存中，不适用于数据量大的情况。会浪费内存，效率低下
* gnerator expression与之等价，但不会缓存结果。

In [12]:
import six
import time

def timer(func):
    @six.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        func()
        print('Running ', str(time.time() - start))
    return wrapper

In [13]:
@timer
def list_comprehension():
    [len(x) for x in open('sample', 'r', encoding='utf-8')]

In [14]:
@timer
def generator_expresion():
    (len(x) for x in open('sample', 'r', encoding='utf-8'))    # generator expresion use small bracket 

In [15]:
print("Generator timing: ")
generator_expresion()
print("list comprehension timing: ")
# list_comprehension()

Generator timing: 
Running  0.0009722709655761719
list comprehension timing: 


* list comprehension 的完整写法

In [16]:
symbols = [1,2,3,4,5]
codes = []
for symbol in symbols:
    codes.append(symbol)

* generator expression的完整写法？

In [17]:
def iter(symbols):
    for symbol in symbols:
        yield symbol
[x for x in iter(symbols)]

[1, 2, 3, 4, 5]

### Term 16 - Iterator （遍历）
考虑用generator改写直接返回列表的函数。
list，set，等是可迭代对象。而generator是返回一个迭代器。我们可以利用迭代器来代替list，优点是节省内存

### Term 17 - Iterator （遍历）
当用generator改写返回列表的函数之后，注意，generator是有状态的，只能遍历一次。

In [44]:
def normalize(numbers):
    total = sum(numbers)
    for number in numbers:
        yield round(number/total, 2)

In [45]:
print([x for x in normalize([1,2,3,4,5])])

[0.07, 0.13, 0.2, 0.27, 0.33]


### Generator, Iterator
#### Iterator pattern
* 按需一次获取一个数据项。iterator pattern是一种范式，对应于c++中的基于范式的编程。即只要程序/类实现了某个pattern的接口，那它就是那个pattern的一种范式的实现
* \__iter\__是与之对应的协议，与接口概念相同，但不强制实现

In [18]:
import re

RE_WORD = re.compile('\w+')


class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words =  RE_WORD.findall(text)
    
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)

* Sentence类是可以迭代的。因为它实现了 **\__getitem\__** 
* 可以迭代的定义：
    * 首先如果有 \__iter\__ 调用 iter
    * 没有iter，调用\__getitem\__, 从index=0 开始迭代到最后， 即 catch StopIteration
    * 还没有，raise TypeError 

In [19]:
s = Sentence('The world’s biggest electric firms have good reason to be focused on China.')
print([x for x in s])

['The', 'world', 's', 'biggest', 'electric', 'firms', 'have', 'good', 'reason', 'to', 'be', 'focused', 'on', 'China']


##### 可迭代对象 iterable vs 迭代器 iterator
* 可迭代对象实现\__iter\__ or \__getitem\__ 接口
* 这些接口，可以返回一个iterator
* Iterator是这样的对象
    * 实现了next()
    * 没有元素抛出 StopIteration

#### Generator
* 首先generator实现了iterator的所有接口，它是一种迭代器
* 即实现next(), raise StopIteration
* 所谓生成器，就是用yield关键字，把它所在的函数/类变成一个**迭代器**。 在对其next()的时候，产生并遍历值
* 简单的说，yield即返回一个迭代器

In [54]:
class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words =  RE_WORD.findall(text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

### Term 10 - enumerate （遍历）
尽量用enumerate取代range

In [55]:
a = [1,2,3,4,5]
for i in range(len(a)):
    a[i]

* better use enumerate. enumerate产生一个generator，并且之后返回两个值，下标，数值

In [56]:
for index, value in enumerate(a):
    index, value

### Term 11 - zip （遍历）
用Zip遍历多个序列
* py3
    * 把两个以上的iterator封装成生成器，以便稍后求值。该生成器，后面会返回元组
* py2
    * zip不是生成器而是迭代器，平行遍历所有iterator一遍，将产生值汇聚成元组。然后把元组构成的完整列表返回给zip caller，浪费内存

* 如果两个列表长度不同，则遍历会提前结束
    * 可以选择py3 itertools.zip_longest
    * py2 itertools.izip_longest 来选择遍历最长的结束

In [145]:
from six.moves import zip, zip_longest    # py23 zip 处理不同
for a, b in zip([1,2], [1,2,3]):
    print(a,b)

1 1
2 2


In [65]:
for a, b in zip_longest([1,2], [1,2,3]):
    print(a,b)

1 1
2 2
None 3


### Term 12 - 程序控制语句
不要在for，while之后加else，可读性差，容易引起误解
* Python 支持一种特殊语法，在for，while后加else
* 表示，如果break 提前结束，则**不**进入else
* 如果for的iterator为空，进入else

In [71]:
for i in []:
    pass
else:
    print("empty")

empty


In [70]:
for i in [1,2,3]:
    pass
else:
    print("empty")

empty


### Term 13 - 程序控制语句

学会使用try/except/else/finally
* 用try/finally来清理文件句柄，既保证了exception会被raise，又确保资源会被释放

In [93]:
handle = open('sample', 'r', encoding='utf-8')     #py23, 注意py3打开文件如果不specify encoding，中文按gbk来
try:
    data = handle.read()
finally:
    handle.close()

* 用try/except/else 减少try中语句，增加可读性, else是没有异常则进入

In [87]:
import json

def load_json(data, key):
    try:
        result = json.loads(data)
    except ValueError as e:
        raise KeyError from e
    else:
        return result[key]          # still could raise keyError

In [90]:
data = '{"key":"my_key"}'
# print(load_json(data, 'key'))
# load_json(data, 'not_exist')
# load_json('', 'key')

## Part II. 函数
### Term 14
尽量用异常返回特殊状况，而不是None。

None有特殊含义，调用者容易处理出错

### Term 15 （函数式)


In [115]:
def sort_proirity(values, group):
    def helper(x):
        if x in group:       # Note, why helper could access group
            return (0, x)    # compare by tuple, first compare 1st ele, then compare 2nd
        return (1, x)
    values.sort(key=helper)
    _helper_func = helper

In [116]:
data = [8, 3, 1, 2, 5]
sort_proirity(data, group=[3,2])
print(data)

[2, 3, 1, 5, 8]


In [120]:
dir(sort_proirity)
print(sort_proirity.__closure__)

None


### 闭包 closure

示例做一个Averager，计算平均值。
* Method 1：用类实现，所有数值按辅助类的属性保存

In [149]:
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)

In [126]:
avg = Averager()
print(avg(10))
print(avg(11))

10.0
10.5


* Method 2：函数式实现，利用高阶函数make_averager，(高阶函数就是返回值是一个函数）
    * 做的事情实际上，是利用高阶函数，做了一个闭包，把函数 average 和 series[] 绑定在一个闭包里， 最后返回闭包

In [129]:
def make_averager():
    series = []
    def average(new_value):
        series.append(new_value)
        return sum(series)/len(series)
    return average   # 函数 average 和 series[] 绑定在一个闭包里， 最后返回闭包

In [144]:
avg = make_averager()  # 是利用高阶函数，做了一个闭包
print(avg(10))
print(avg(11))

10.0
10.5


In [146]:
avg.__code__.co_varnames

('new_value',)

* 闭包的自由变量 free variable, 指未在本地作用域中绑定的变量

In [147]:
avg.__code__.co_freevars

('series',)

In [141]:
avg.__closure__

(<cell at 0x0000021805E69C78: list object at 0x0000021805E0E708>,)

* \__closure\__中的cell_contents 属性，保存着真正的值

In [148]:
avg.__closure__[0].cell_contents

[10, 11]

### Non Local

### 装饰器 Decorator
装饰器只是语法糖，最简单的调用就是，接受一个函数为参数。
* 一大特性是能把被装饰的函数**替换**成其他函数。（deco 接受target 为参数，返回inner，替换target）
* 另一个特性是，装饰器在模块加载（import）时候运行

In [164]:
def deco(func):
    def inner():
        print("running inner")
    return inner

@deco
def target():
    print("running target")

target()

running inner


* 实际上，装饰器是把target函数，绑定到一个闭包，闭包中可以定义一些额外的操作（比如，增加一个timer，http 注册一个网址）

In [172]:
from functools import wraps
import time

def timer(func):
    
    @wraps(func)                          # 有没有这句话都可以运行，作用如下
    def wrapper(*args, **kwargs):
        start = time.time()
        func()
        print("Running Time ", time.time() - start)
    return wrapper

*  如果没有，则wrapper不支持关键字参数，而且遮盖了target函数的\__name\__和\__doc\__属性，functools.wrap可以把装饰器相关属性从func复制到wrapper中

In [ ]:
@timer
def sleep():
    time.sleep(2)
sleep()

#### 装饰器的一些高级应用
##### Singleddispatch
python不支持函数重载，对于一些类型不同，处理方式不同的函数，很头疼，python3.4引入singledispatch
* htmlize.register 装饰器，把多个函数绑在一起组成一个泛函数

In [196]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):                                       # 函数名称不重要，基于参数类型register， then dispatch
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '<li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '<li>\n<ul>'

In [193]:
print('String: ', htmlize('abc'))
print('Number: ', htmlize(123))
print('Sequence:\n', htmlize(('a','b','c')))
print('not specified handler, for dict: ', htmlize({'a':1,'b':2}))

String:  <p>abc</p>
Number:  <pre>123 (0x7b)</pre>
Sequence:
 <ul>
<li><p>a</p><li>
<li><p>b</p><li>
<li><p>c</p><li>
<ul>
not specified handler, for dict:  <pre>{&#x27;a&#x27;: 1, &#x27;b&#x27;: 2}</pre>


#### 叠放装饰器

#### 参数化装饰器

In [231]:
registry = set()

def register(active=True):
    
    def decorate(func):
        if active:
            print('running register(%s)' % func)
            registry.add(func)
        else:
            print('discard (%s)' % func)
            registry.discard(func)
        
        return func
    
    return decorate

@register()
def f1():
    print('running f1')

print('First register f1: ', registry)

running register(<function f1 at 0x0000021803AA2950>)
First register f1:  {<function f1 at 0x0000021803AA2950>}


In [232]:
register(active=False)(f1)

print('Discard f1: ', registry)

discard (<function f1 at 0x0000021803AA2950>)
Discard f1:  set()


##### Part III. 类
###### Term 23 - 接口
简单的接口应该接受函数，而不是类的实例

In [104]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

result = defaultdict(log_missing)    # 所以给它int，list，dict 实际上是传递了一个构造函数，如果没有调用返回默认值
result['A'] += 1 

Key added


##### 多态
* mapReduce示例，定义InputData，Worker两个基类。InputData定义了read()接口，Worker定义了map(),reduce()接口。
* 问题是，我们需要一些辅助函数gnerate_input，create_worker etc把功能链接起来
* generate_input是和InputData绑定的，不同的类别有自己generate_input的方法，需要多态
* create_worker是统一的。我们不想为了每一个InputData重写worker level的辅助函数

###### @classmethod

In [14]:
# classmethod passed in class, rather then instance
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

* passed in class

In [11]:
Demo.klassmeth()

(__main__.Demo,)

* staticmethod passed in only arguments

In [12]:
Demo.statmeth()

()

In [13]:
Demo.statmeth('spma')

('spma',)

* 该例中classmethod起到了工厂函数的作用，工厂函数是要产生对象，所以必须在类方法上定义，而非实例方法。

In [56]:
import os
class InputData(object):
    def read(self):
        raise NotImplementedError
    
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError
        
class PathInputData(object):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        return open(self.path, 'rb').read().decode('utf-8')
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [61]:
class Worker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
        
    def reduce(self):
        raise NotImplementedError
        
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

In [62]:
config = {'data_dir':'sample'}
workers = LineCountWorker.create_workers(PathInputData, config)

sample test
sample test2
sample test3


In [69]:
workers[1].map()
workers[1].reduce(workers[0])
print(workers[1].result)

2


##### 2. classmethod实现备选构造方法

In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(array(self.typecode, self)))
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    # make vector2d hashable 
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

* 可散列的对象必须实现\__hash\__方法 和\__eq\__
* 保证在 a== b 为真的情况下 hash(a) == hash(b) 也必定为真。否则就会破坏恒定的散列表算法，导致由这些对象所组成的字典和集合完全失去可靠性，这个后果是非常可怕的。

In [2]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

In [3]:
print(hash(v1), hash(v2))
set([v1, v2])

(7, 537100288)


{instance(3.0, 4.0), instance(3.1, 4.2)}

* \__repr\__ 方法使用 **{!r}** 获取各个分量的表示形式，然后插值，构成一个字符串

In [33]:
v = Vector2d(1, 2)
print('test __iter__: ', [x for x in v])
print('test __repr__: ', v)

test __iter__:  [1.0, 2.0]
test __repr__:  Vector2d(1.0, 2.0)


In [34]:
print('test __bytes__: ', bytes(v2))

test __bytes__:  b'd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [ ]:
v2 = Vector2d(0,0)
print('test __abs__: ', abs(v))
print('test __abs__: ', abs(v2))

* python不支持函数重载，所以我们不能重载构造函数
* classmethod的trick是，重新命名一个函数名，但是可以把类对象给它，变相的实现的重载的功能

###### repr vs str
* repr() 以便于开发者理解的方式返回对象的字符串表示形式。
* str() 以便于用户理解的方式返回对象的字符串表示形式。

###### Great example for bytes repr
* chr()
* octets
* memoryview
* typecode

In [40]:
bytes_repr = bytes(v)
constructed_from_bytes = Vector2d.frombytes(bytes_repr)
print("test classmethod: ", constructed_from_bytes)

test classmethod:  Vector2d(1.0, 2.0)


### 序列
###### 容器序列
* list、tuple 和 collections.deque 这些序列能存放不同类型的数据
* 容器序列里储存的是对象引用

###### 扁平序列
* str、bytes、bytearray、memoryview 和 array.array，这类序列只能容纳一种类型
* 扁平序列里储存的是值

###### 可变序列
* list、bytearray、array.array、collections.deque 和 memoryview

###### 不可变序列
* tuple、str 和 bytes

### 文本和字节序列
fluent python, 正确理解字符，人类可读的字节序列，字符编码，字节，字节流，解码

* 人类可读的字节序列
* unicode字符串
* 二进制序列：bytes, bytearray, memoryview


In [68]:
def test_encoding(uni):
    unicodes_ = uni
    bytes_ = unicodes_.encode("utf-8")
    print("bytes:", type(bytes_), bytes_)
    print("unicodes: ", type(unicodes_), unicodes_)

In [71]:
test_encoding('中国')

bytes: <class 'bytes'> b'\xe4\xb8\xad\xe5\x9b\xbd'
unicodes:  <class 'str'> 中国


In [70]:
test_encoding('China')

bytes: <class 'bytes'> b'China'
unicodes:  <class 'str'> China


##### bypes vs bytearray
* Python 3引入的不可变bytes类型和Python 2.6添加的bytearray可变类型。
* bytes或bytearray对象的各个元素是介于0~255（含）之间的整数

In [105]:
cafe = bytes('caf', encoding='utf_8')
cafe_arr = bytearray(cafe)
cafe_arr += bytearray(b'\xc3\xa9')

print(cafe_arr[0], cafe_arr[:1])       # bytearray[0]取得的是一个整数，bytearray[:1]是一个切片

99 bytearray(b'c')


In [93]:
print(cafe_arr)
print(cafe_arr.decode('utf-8'))

bytearray(b'caf\xc3\xa9')
café
99 bytearray(b'c')


#### encoding and files

In [109]:
open('sample/cafe.txt', 'w', encoding='utf-8').write('café')

4

In [110]:
open('sample/cafe.txt').read()

'caf茅'

In [120]:
 open('sample/cafe.txt', encoding='utf-8').read()    

'café'

##### python 2, 3
* py23, python2 str is bytes, but python3 str is unicode. So how to write a open function work for both
* 所以用‘rb’ option强制读出的是bytes而不是unicode，再多bytes解码

In [119]:
open('sample/cafe.txt', 'rb').read().decode('utf-8')

'café'

#### Term 25 用super初始化父类 (继承）
* 一般继承方法，再子类里调用父类的\__init\__
* super
    * python 2：super(Child, self).\__init\__()
    * python 3: super().\__init\__()
    
###### 继承中的常见问题
* 多重继承，初始化顺序不确定

In [165]:
class BaseA(object):
    def __init__(self):
        print('Initialized Base A')

class BaseB(object):
    def __init__(self):
        print('Initialized Base B')

class Child(BaseA, BaseB):
    pass
        
class Child2(BaseB, BaseA):
    pass

##### MRO
* method resolution order, 调用这个方法可以看到规定的标准化的类的初始化顺序
* Child，Child2由于签名写的不同，父类的初始化顺序不同

In [166]:
print(Child.mro())
print(Child())

[<class '__main__.Child'>, <class '__main__.BaseA'>, <class '__main__.BaseB'>, <class 'object'>]
Initialized Base A


In [167]:
print(Child2.mro())
print(Child2())

[<class '__main__.Child2'>, <class '__main__.BaseB'>, <class '__main__.BaseA'>, <class 'object'>]
Initialized Base B


#### Diamond 
* 钻石形状继承，会让公共基类多次初始化

In [206]:
class Base(object):
    data = 0
    def __init__(self, value):
        print('Base Constractor called')
        self.data += value

class BaseC(Base):
    def __init__(self, value):
        Base.__init__(self, value)

class BaseD(Base):
    def __init__(self, value):
        Base.__init__(self, value)

class Plus2(BaseC, BaseD):
    def __init__(self, value):
        BaseC.__init__(self, value)
        BaseD.__init__(self, value)

In [207]:
print(Plus2.mro())
p = Plus2(2)
print(p.data)

[<class '__main__.Plus2'>, <class '__main__.BaseC'>, <class '__main__.BaseD'>, <class '__main__.Base'>, <class 'object'>]
Base Constractor called
Base Constractor called
4


* 使用super，公共基类只会初始化一次

In [208]:
class BaseE(Base):
    def __init__(self, value):
        print('BaseC Constractor called')
        super().__init__(value)

class BaseF(Base):
    def __init__(self, value):
        print('BaseD Constractor called')
        super().__init__(value)

class Plus2D(BaseE, BaseF):
    def __init__(self, value):
        super().__init__(value)

In [209]:
p = Plus2D(2)
print(p.data)

BaseC Constractor called
BaseD Constractor called
Base Constractor called
2


* super在parent signature不明确的时候会造成混乱，根据MRO顺序进行初始化，但是参数怎么分配呢

In [249]:
class BaseE(Base):
    def __init__(self, a, b):
        print('BaseC Constractor called')
        super().__init__(a)
        self.b = b

class BaseF(Base):
    def __init__(self, a, d):
        print('BaseD Constractor called')
        super().__init__(a)

class Thrid(BaseE, BaseF):
    def __init__(self, a, b):
        super().__init__(a, b)

In [250]:
Thrid(1,3)

BaseC Constractor called


TypeError: __init__() missing 1 required positional argument: 'd'

In [213]:
t.data, t.b

(2, 3)

###### Mix-in

In [260]:
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for k, v in instance_dict.items():
            output[k] = self._traverse(k, v)
        return output
    
    def _traverse(self, k, v):
        if isinstance(v, ToDictMixin):
            return v.to_dict()
        elif isinstance(v, dict):
            return self._traverse_dict(v)
        elif isinstance(v, list):
            return [self._traverse(k, i) for i in value]
        elif hasattr(v, '__dict__'):
            return self._traverse_dict(v.__dict__)
        else:
            return v
        
class BinaryTree(ToDictMixin):
    
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [262]:
t = BinaryTree(10, left=BinaryTree(7, right=BinaryTree(9)), right=BinaryTree(13))
print(t.to_dict())

{'value': 10, 'right': {'value': 13, 'right': None, 'left': None}, 'left': {'value': 7, 'right': {'value': 9, 'right': None, 'left': None}, 'left': None}}


#### Public vs private
如果以 \__mood 的形式（两个前导下划线，尾部没有或最多有一个下划线）命名实例属性，Python 会把属性名存入实例的\__dict\__ 属性中，而且会在前面加上一个下划线和类名。因此，对Dog 类来说，\__mood 会变成 _Dog\__mood；对 Beagle 类来说，会变成 _Beagle\__mood。这个语言特性叫名称改写（name mangling）
* 但是python只是改名，用户如果知道规则也是可以访问

In [12]:
class MyObject(object):
    
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
    
    @property
    def field(self):
        return self.__private_field

* 直接访问私有属性会报错

In [13]:
MyObject().__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

* 用户通过改名的手段，来访问

In [24]:
MyObject()._MyObject__private_field

10

* 类自己的方法可以访问私有属性

In [15]:
MyObject().field

10

* 私有属性不能继承

In [16]:
class ChildObject(MyObject):
    def get_parent_private_field(self):
        return self.__private_field

In [9]:
ChildObject().get_parent_private_field()

AttributeError: 'ChildObject' object has no attribute '_ChildObject__private_field'

* hacky way to get parent class private attributes

In [11]:
ChildObject()._MyObject__private_field

10

##### abc，Abstract Base Class 定义抽象基类来规定子类必须实现的方法

In [22]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

In [23]:
foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

#### abc vs duck typing
* 协议是接口，但不是正式的（只由文档和约定定义）
* 因此协议不能像正式接口那样施加限制, 抽象基类对接口一致性的强制
* 一个类可能只实现部分接口，这是允许的

In [28]:
Sequence.__contains__

<function collections.abc.Sequence.__contains__>

In [29]:
Sequence.__iter__

<function collections.abc.Sequence.__iter__>

In [30]:
Sequence.__len__

<function collections.abc.Sized.__len__>

### Inheritance - Py23 ???

In [67]:
from abc import ABC, ABCMeta

class MyABC(metaclass=ABCMeta):
    pass

In [68]:
MyABC.register(tuple)

tuple

In [69]:
assert issubclass(tuple, MyABC)

In [70]:
MyABC()

In [15]:
class MyClass():

    def __init__(self, data):
        self.data = data

    def __str__(self):
        return str(self.data)

    def __len__(self):
        return len(self.data)

In [16]:
ins = MyClass([1, 2, 3])

In [17]:
print(ins)

[1, 2, 3]


In [18]:
len(ins)

3